# Transformers from Scratch
This is a workthrough of [the nice derivation found here.](https://e2eml.school/transformers.html#table_lookup)

# Setup

## Imports

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

## Global Variables

In [ ]:
rng = np.random.default_rng()

# Contents

## One-hot encoding

* Words -> numerical representation.

Consider "find my files":

\begin{matrix}
\rm{files} & \rm{find} & \rm{my}
\end{matrix}

\begin{pmatrix}
0 & 1 & 0 \\
0 & 0 & 1 \\
1 & 0 & 0
\end{pmatrix}


In [ ]:
# Pandas one-hot encoding
phrase = pd.Series(['find', 'my', 'files'])
encoded_phrase = pd.get_dummies(phrase) # one-hot encoding
encoded_phrase.astype(int)

## Dot product

$\vec{a} \cdot \vec{b} \equiv a_i b_i = c$ (scalar)

## Matrix multiplication

$\bf{A} \bf{B} \equiv A_{ij}B_{jk} = C_{ik}$

## Matrix multiplication as a table lookup

In [ ]:
some_matrix = rng.uniform(size=(3, 3))
some_matrix

In [ ]:
one_hot_vec = encoded_phrase['files']
np.matmul(some_matrix, one_hot_vec) # Last column

## First order sequence model

In [ ]:
def simple_tokenize(text):
    return text.split()

In [ ]:
phrases = [
    'show me my directories please',
    'show me my files please',
    'show me my photos please',
]
tokenized_phrases = [simple_tokenize(phrase) for phrase in phrases]
tokenized_phrases

In [ ]:
def build_vocab(tokenized_phrases):

    # Get all tokens
    tokens = set()
    for tokens_i in tokenized_phrases:
        tokens.update(tokens_i)

    # Convert to dictionary
    vocab = {}
    for i, token in enumerate(sorted(tokens)):
        vocab[token] = i

    return pd.Series(vocab)

In [ ]:
vocab = build_vocab(tokenized_phrases)
vocab

In [ ]:
def build_transition_df(tokenized_phrases, vocab, h=1, verbose=True):

    transition_mat = np.zeros((len(vocab), len(vocab))).astype(int)

    # Loop through and add
    for ii, tokens_ii in enumerate(tokenized_phrases):
        for jj, word in enumerate(tokens_ii):
            start = max(0, jj - h)
            for k, other_word in enumerate(tokens_ii[start:jj]):

                if verbose:
                    print(f'({start}, {jj}): {other_word} -> {word}')

                i = vocab.loc[other_word]
                j = vocab.loc[word]
                transition_mat[i, j] += 1

    transition_df = pd.DataFrame(
        transition_mat,
        index=vocab.index,
        columns=vocab.index
    )

    return transition_df

In [ ]:
transition_df = build_transition_df(tokenized_phrases, vocab)
transition_df

In [ ]:
# Probabilities
transition_df.div(transition_df.sum(axis=1), axis=0)

In [ ]:
def vocab_to_one_hot(vocab):
    mat = np.identity(len(vocab), dtype=int)
    return pd.DataFrame(mat, index=vocab.index)

In [ ]:
vocab_identity = vocab_to_one_hot(vocab)
vocab_identity

In [ ]:
np.matmul(transition_df.T.values, vocab_identity.loc['my'])

## Second order sequence model

In [ ]:
phrases = [
    'check whether the battery ran down please',
    'check whether the program ran please'
]
tokenized_phrases = [simple_tokenize(phrase) for phrase in phrases]
tokenized_phrases

In [ ]:
def build_ngram_vocab(phrases, *args, **kwargs):

    vectorizer = CountVectorizer(*args, **kwargs)
    X = vectorizer.fit_transform(phrases)
    vectorized_phrases = pd.DataFrame(
        X.toarray(),
        columns=vectorizer.get_feature_names_out(),
    )

    vocab = pd.Series(vectorizer.vocabulary_).sort_values()

    return vectorized_phrases, vocab


In [ ]:
bigram_phrases, bigram_vocab = build_ngram_vocab(phrases, ngram_range=(2, 2))
bigram_phrases

In [ ]:
vocab = build_vocab(tokenized_phrases)

In [ ]:
vocab, bigram_vocab

In [ ]:
def build_bigram_transition_df(
    tokenized_phrases,
    vocab,
    bigram_vocab,
    verbose=True,
):

    transition_mat = np.zeros((len(bigram_vocab), len(vocab))).astype(int)

    # Loop through and add
    for ii, tokens_ii in enumerate(tokenized_phrases):
        for jj, word in enumerate(tokens_ii):

            # No bigram for first word
            if jj < 2:
                continue

            preceeding_bigram = ' '.join([tokens_ii[jj - 2], tokens_ii[jj - 1]])

            i = bigram_vocab.loc[preceeding_bigram]
            j = vocab.loc[word]

            if verbose:
                print(f'({jj-2} + {jj-1}, {jj}): {preceeding_bigram} -> {word} @ {i}, {j}')

            transition_mat[i, j] += 1

    transition_df = pd.DataFrame(
        transition_mat,
        index=bigram_vocab.index,
        columns=vocab.index
    )

    return transition_df

In [ ]:
build_bigram_transition_df(tokenized_phrases, vocab, bigram_vocab)

## Second order sequence model with skips

In [ ]:
phrases = [
    'check the program log and find out whether it ran please',
    'check the battery log and find out whether it ran down please',
]
tokenized_phrases = [simple_tokenize(phrase) for phrase in phrases]
tokenized_phrases

In [ ]:
vocab = build_vocab(tokenized_phrases)
vocab

In [ ]:
transition_df = build_transition_df(tokenized_phrases, vocab, h=100)
transition_df